优化算法:使用Keras提供的其他优化器，如梯度下降，看在其他算法下模型参数对模型训练和过拟合的速度有怎样的影响。
损失函数:尝试使用Keras其他可用的损失函数，探究选用其他的损失函数是否可以提升模型的性能。
学习率与迭代次数更新策略
更大的Batch Size:使用更大的Batch Size意味着模型在训练集和测试集上的数据操作规模更大了

In [ ]:
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.applications.inception_v3 import preprocess_input
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from keras.layers import *
#from keras.layers import Input
from keras.models import *
#from keras.models import Model
#from keras.layers import Input, Dense
#a = Input(shape=(32,))
#b = Dense(32)(a)
#model = Model(inputs=a, outputs=b)
from keras.callbacks import *
from keras.optimizers import *
from keras.regularizers import *
from keras import initializers
from keras.applications import *

#from keras.utils import multi_gpu_model 
#from keras.applications import ResNet50
#from keras.applications import VGG16
#from keras.applications import VGG19
#from keras.applications import Xception # TensorFlow ONLY
#from keras.applications import InceptionResNetV2
#from keras.applications import InceptionV3

#tf.keras.applications.inception_v3.InceptionV3
#tf.keras.applications.inception_resnet_v2.InceptionResNetV2
####################################################################
#设置GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

#设置项
#看具体的模型参数设置在:https://www.tensorflow.org/api_docs/python/tf/keras/applications
MODELS = {"InceptionResNetV2":InceptionResNetV2}
#MODELS = {"NASNetLarge":NASNetLarge,"VGG16":VGG16}
#"InceptionV3":InceptionV3,"DenseNet121":DenseNet121,
 #       "DenseNet169":DenseNet169,"DenseNet201":DenseNet201,"Xception":Xception, 
 #       "InceptionResNetV2":InceptionResNetV2,
#"ResNet50":ResNet50, 
#"VGG16":VGG16,"VGG16":VGG19,"NASNetMobile":NASNetMobile

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

classes = ['coat_length_labels','collar_design_labels', 'lapel_design_labels',
           'neck_design_labels','neckline_design_labels',
           'pant_length_labels', 'skirt_length_labels', 
           'sleeve_length_labels']    
fai_result = []
label_count = {'coat_length_labels':8,
               'collar_design_labels':5, 
               'lapel_design_labels':5,
               'neck_design_labels':5,
               'neckline_design_labels':10,
               'pant_length_labels':6, 
               'skirt_length_labels':6, 
               'sleeve_length_labels':9}
#####################
width = 299
#####################


df_train = pd.read_csv('../train/Annotations/train.csv', header=None)

df_train.columns = ['image_id', 'class', 'label']
df_load = df_train.copy()
df_load.reset_index(inplace=True)
del df_load['index']
#样本总数 
n = len(df_load)
X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, 54), dtype=np.uint8)

for i in tqdm(range(n)):
    tmp_label = df_load['label'][i]
    n_class = len(df_load['label'][0])
    if len(tmp_label) == label_count[df_load['class'][i]]:
        
        X[i] = cv2.resize(cv2.imread('../train/{0}'.format(df_load['image_id'][i])), (width, width))
        if df_load['class'][i] == 'coat_length_labels':
            y[i][tmp_label.find('y')] = 1
        elif df_load['class'][i] == 'collar_design_labels':
            kk = y[i][tmp_label.find('y')] + 8
            y[i][kk] = 1
        elif df_load['class'][i] == 'lapel_design_labels':
            kk = y[i][tmp_label.find('y')] + 13
            y[i][kk] = 1
        elif df_load['class'][i] == 'neck_design_labels':
            kk = y[i][tmp_label.find('y')] + 18
            y[i][kk] = 1
        elif df_load['class'][i] == 'neckline_design_labels':
            kk = y[i][tmp_label.find('y')] + 23
            y[i][kk] = 1
        elif df_load['class'][i] == 'pant_length_labels':
            kk = y[i][tmp_label.find('y')] + 33
            y[i][kk] = 1
        elif df_load['class'][i] == 'skirt_length_labels':
            kk = y[i][tmp_label.find('y')] + 39
            y[i][kk] = 1
        else:
            kk = y[i][tmp_label.find('y')] + 45
            y[i][kk] = 1          
print("数据装载到内存完毕:{0}".format("label:54-one-hot,下面训练一个分类器"))

In [ ]:
for KEY, MODLE in MODELS.items():
    #
    n_class = 54
    #为299*299,设置如下
    ppreprocess = preprocess_input
    if KEY in ["InceptionV3","Xception", "InceptionResNetV2"]:
        width = 299
    elif KEY == "NASNetLarge":
        width = 331
    else:
        width = 224
        ppreprocess = imagenet_utils.preprocess_input 
    print('######################在{0}下训练8个分类器####################'.format(KEY))
    cnn_model = MODLE(include_top=False, input_shape=(width, width, 3), weights='imagenet',pooling='avg')
    inputs = Input((width, width, 3))
    x = inputs
    x = Lambda(ppreprocess, name='preprocessing')(x)
    x = cnn_model(x)
    #下面是新加的层
    #x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    #x = Flatten(name='flatten')(x)
    #x = Dense(2048, activation='relu', name='fc1')(x)
    # n_class为对应属性的分类个数
    #x = Dense(256, activation='relu', kernel_initializer=initializers.he_uniform(seed=None),name='fc1')(x)
    x = Dense(n_class, activation='softmax', name='softmax')(x)

    model = Model(inputs, x)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.12, random_state=50)
        
    #sgd = SGD(lr=learning_rate, decay=learning_rate/nb_epoch, momentum=0.9, nesterov=True)
    #adam = optimizers.Adam(lr=1e-4)
    #optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))
    adam = Adam(lr=0.001)
        
    #多GPU训练,因为keras设计的自动保存最好模型,但是多GPU训练,其save()就没法用了
    #model = multi_gpu_model(model, 2)  

    model.compile(optimizer=adam,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# Callback that implements learning rate schedule
#schedule = Step([20], [1e-4, 1e-6])
#history = model.fit(X_train, Y_train,
#                    batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(X_test,Y_test),
#                    callbacks=[
#                           schedule,
#                           keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0,save_best_only=True, mode='auto')

# 该回调函数将在每个epoch后保存模型到filepath
#keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0,save_best_only=True, mode='auto')
# 当监测值不再改善时，该回调函数将中止训练.
#当early stop被激活（如发现loss相比上一个epoch训练没有下降），则经过patience个epoch后停止训练
#keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

    #设置训练完之后,最好的模型保存路径
    #checkpointer = ModelCheckpoint(filepath='../models/{0}.best.h5'.format(KEY), verbose=1, 
                                  #  save_best_only=True)
    #训练开始,并保存训练过程的loss和acc变化
    h = model.fit(X_train, y_train, batch_size=6, epochs=8, 
                     # callbacks=[EarlyStopping(patience=10), checkpointer], 
                      shuffle=True, 
                      validation_data=(X_valid,y_valid))
    #score = model.evaluate(X_valid,y_valid,batch_size=64,verbose=0)
    #print ('{0}验证平均accuracy:{1}'.format(KEY,score[1]))
    
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(h.history['loss'])
    plt.plot(h.history['val_loss'])
    plt.legend(['loss', 'val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.title('{0}_loss'.format(KEY))
        
    plt.subplot(1, 2, 2)
    plt.plot(h.history['acc'])
    plt.plot(h.history['val_acc'])
    plt.legend(['acc', 'val_acc'])
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.title('{0}_accuracy'.format(KEY))
    #保存训练损失和准确率变化的图像
    plt.savefig('../models/{0}v.png'.format(KEY),bbox_inches='tight')
        
    

In [ ]:
 
#测试集上预测并输出结果
df_test = pd.read_csv('../test/Tests/question.csv', header=None)
df_test.columns = ['image_id', 'class', 'x']
del df_test['x']  
df_load = df_test.copy()
df_load.reset_index(inplace=True)
del df_load['index']  
n = len(df_load)
X_test = np.zeros((n, width, width, 3), dtype=np.uint8)

for i in range(n):
    
    X_test[i] = cv2.resize(cv2.imread('../test/{0}'.format(df_load['image_id'][i])), (width, width))

test_np = model.predict(X_test, batch_size=256)
print(test_np)
result = []
for i, row in df_load.iterrows():
    tmp_list = test_np[i]
    tmp_label = df_load['label'][i]
    tmp_result = ""
    if df_load['class'][i] == 'coat_length_labels':
        dd = 0
        aa = 8
    elif df_load['class'][i] == 'collar_design_labels':
        dd = 8
        aa = 13
    elif df_load['class'][i] == 'lapel_design_labels':
        dd = 13
        aa = 18
    elif df_load['class'][i] == 'neck_design_labels':
        dd = 18
        aa = 23
    elif df_load['class'][i] == 'neckline_design_labels':
        dd = 23
    elif df_load['class'][i] == 'pant_length_labels':
        dd = 33
        aa = 39
    elif df_load['class'][i] == 'skirt_length_labels':
        dd = 39
        aa = 45
    else:
        dd = 45 
        aa = 54
    tmp_listdd = tmp_list[dd:aa]
    for tmp_ret in tmp_listdd:
            
        tmp_result += '{:.4f};'.format(tmp_ret)

    result.append(tmp_result[:-1])

df_load['result'] = result     
df_load.to_csv('../result/{0}/{0}.csv'.format(KEY), header=None, index=False)
print(fai_result)

In [ ]:
print(fai_result)